In [1]:
!pip install sentence-transformers

In [2]:
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, util , SentencesDataset
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import torch
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import random

/Users/madness/Desktop/manga-recommendation/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv("data/df.csv")

In [5]:
n = 5000

In [6]:
all_q = []
all_des = []
all_title = []
all_index = []

for i in range(n):
    if("(Novel)" in df.loc[i, "title"] or "(Light Novel)" in df.loc[i, "title"] or "(Webcomic)" in df.loc[i, "title"] or "(Doujinshi)" in df.loc[i, "title"]):
        continue
    if(df.loc[i, "q"]):
        all_q.append(df.loc[i, "q"].split('\n')[0])
        all_des.append(str(df.loc[i, "description"]))
        all_title.append(str(df.loc[i, "title"]))
        all_index.append(i)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(all_q, all_des, test_size=0.25, random_state=42)

In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [9]:
train_examples = []
for i in range(len(x_train)):
  train_examples.append(InputExample(texts=[str(x_train[i]), str(y_train[i])], label=1.0))
# make unrelated data
for i in range(len(x_train)):
  rand_idx_des = random.randint(0, len(x_train)-1)
  rand_idx_q = random.randint(0, len(y_train)-1)
  while(rand_idx_des == rand_idx_q):
    rand_idx_q = random.randint(0, len(y_train)-1)
  train_examples.append(InputExample(texts=[str(x_train[rand_idx_des]), str(y_train[rand_idx_q])], label=0.0))

In [10]:
len(train_examples)

6358

In [11]:
train_dataloader = DataLoader(train_examples, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

In [12]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
        epochs=5, warmup_steps=100, show_progress_bar=True)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
model.save("./model.pt")

In [12]:
path = './model.pt'
model = SentenceTransformer(path)

In [ ]:
passage_embedding = model.encode(['Detective', 'นักสืบ'])

In [ ]:
len(passage_embedding[1])

384

In [ ]:
cosine_scores = util.cos_sim(passage_embedding[0], passage_embedding[1])

In [ ]:
cosine_scores[0][0]

tensor(0.6290)

In [13]:
val_q = x_test

In [14]:
embeddings_des = model.encode(all_des, convert_to_tensor=True)

In [15]:
all_val_q = []
label = []
for i in range(len(val_q)):
    embedding_q = model.encode(val_q[i], convert_to_tensor=True)
    idxx = all_q.index(val_q[i])
    label.append(idxx)
    all_val_q.append(embedding_q)

In [22]:
len(embeddings_des)

4239

In [23]:
len(all_val_q)

1060

In [24]:
type(all_val_q), type(all_val_q[0])

(list, torch.Tensor)

In [20]:
acc = 0
n_val = len(all_val_q)
topk = 5

for i in range(n_val):
    cosine_scores = util.cos_sim(all_val_q[i], embeddings_des)
    #print(len(cosine_scores[0]))
    #max_val = max(cosine_scores[0])
    all_idx = torch.topk(cosine_scores[0], topk).indices
    ans = []
    for j in all_idx:
        ans.append(all_index[int(j)])

    #check idex in all_des==idx
    if(all_index[label[i]] in ans):
        acc += 1

In [17]:
print("top1 acc :", acc/len(all_val_q))

top1 acc : 0.0


In [19]:
print("top3 acc :", acc/len(all_val_q))

top3 acc : 0.0


In [21]:
print("top5 acc :", acc/len(all_val_q))

top5 acc : 0.0009433962264150943


#Test test

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import torch

In [ ]:
df = pd.read_csv("df.csv")

In [ ]:
n = 5000

In [ ]:
all_q = []
all_des = []
all_title = []
all_index = []

for i in range(n):
    if("(Novel)" in df.loc[i, "title"] or "(Light Novel)" in df.loc[i, "title"] or "(Webcomic)" in df.loc[i, "title"] or "(Doujinshi)" in df.loc[i, "title"]):
        continue
    if(df.loc[i, "q"]):
        all_q.append(str(df.loc[i, "q"]).split('\n')[0])
        all_des.append(str(df.loc[i, "description"]))
        all_title.append(str(df.loc[i, "title"]))
        all_index.append(i)

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

In [ ]:
embeddings_des = model.encode(all_des, convert_to_tensor=True)

In [ ]:
val_q = all_q

In [ ]:
all_val_q = []
label = []
for i in range(len(val_q)):
    embedding_q = model.encode(val_q[i], convert_to_tensor=True)
    idxx = all_q.index(val_q[i])
    label.append(idxx)
    all_val_q.append(embedding_q)

NameError: ignored

In [ ]:
acc = 0
n_val = len(all_val_q)
topk = 5

for i in range(n_val):
    cosine_scores = util.cos_sim(all_val_q[i], embeddings_des)
    #print(len(cosine_scores[0]))
    #max_val = max(cosine_scores[0])
    all_idx = torch.topk(cosine_scores[0], topk).indices
    ans = []
    for j in all_idx:
        ans.append(all_index[int(j)])

    #check idex in all_des==idx
    if(all_index[label[i]] in ans):
        acc += 1

In [ ]:
print("top1 acc :", acc/len(all_val_q))

top1 acc : 0.4170794998820477


In [ ]:
print("top3 acc :", acc/len(all_val_q))

top3 acc : 0.5119131870724227


In [ ]:
print("top5 acc :", acc/len(all_val_q))

ZeroDivisionError: ignored